# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

### Create Database:

In [2]:
from create_tables import create_database
cur, conn = create_database()

### Or, get DB-Connection

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

### Create Tables, if necessary:

In [4]:
from create_tables import create_tables
create_tables(cur, conn)

### Helper Function:

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data')

In [7]:
song_files

['/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json',
 '/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAAW128F429D538-checkpoint.json',
 '/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAVO128F93133D4-checkpoint.json',
 '/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAVG12903CFA543-checkpoint.json',


In [8]:
filepath = song_files[0]
filepath

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [9]:
df = pd.read_json(filepath, lines=True)
df.head(3)

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
song_data = df[['song_id','title','artist_id', 'year', 'duration']]
song_data

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179


In [11]:
song_data = song_data.values.tolist()[0]
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN


In [14]:
artist_data = artist_data.values.tolist()[0]
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files('data/log_data')

In [17]:
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [18]:
df = pd.read_json(filepath, lines=True)
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [19]:
df.query("page != 'NextSong'").head(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
8,None,Logged In,Alivia,F,0,Terrell,NaN,free,"Parkersburg-Vienna, WV",GET,Home,1.540505e+12,1070,None,200,1543541644796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",4
20,None,Logged In,Jacob,M,18,Klein,NaN,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,Logout,1.540558e+12,1049,None,307,1543544360796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
21,None,Logged Out,None,None,19,None,NaN,paid,None,GET,Home,NaN,1049,None,200,1543544407796,None,
22,None,Logged In,Aiden,M,0,Hess,NaN,free,"La Crosse-Onalaska, WI-MN",GET,Home,1.540829e+12,986,None,200,1543547152796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",86
28,None,Logged In,Layla,F,0,Griffin,NaN,paid,"Lake Havasu City-Kingman, AZ",GET,Home,1.541057e+12,1051,None,200,1543548391796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [20]:
df = df.query("page=='NextSong'")
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


Remember: **Epoch is given here in ms!**

In [21]:
t = df['ts']
t.head(5)

0    1543537327796
1    1543540121796
2    1543540368796
3    1543540625796
4    1543540856796
Name: ts, dtype: int64

In [22]:
time_stamp = pd.Series(pd.to_datetime(t, unit='ms'))
time_stamp.head(5)

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [23]:
# Isn't used below, just as practice:
#epoch = (time_series - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
epoch = t // 1000
epoch.head(5)

0    1543537327
1    1543540121
2    1543540368
3    1543540625
4    1543540856
Name: ts, dtype: int64

In [24]:
time_stamp.dt.week.head()

0    48
1    48
2    48
3    48
4    48
Name: ts, dtype: int64

In [25]:
# timestamp, hour, day, week of year, month, year, and weekday
time_data = (time_stamp, time_stamp.dt.hour, time_stamp.dt.day, time_stamp.dt.week, time_stamp.dt.month, time_stamp.dt.year, time_stamp.dt.weekday)
column_labels = ('timestamp', 'hour', 'day', 'week of year', 'month', 'year', 'weekday')

In [26]:
timedata_dict = dict(zip(column_labels, time_data))

In [27]:
time_df = pd.DataFrame.from_dict(timedata_dict)
time_df.head(8)

,timestamp,hour,day,week of year,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4
5,2018-11-30 01:24:32.796,1,30,48,11,2018,4
6,2018-11-30 01:28:19.796,1,30,48,11,2018,4
7,2018-11-30 01:32:00.796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [28]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [29]:
 df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [30]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head(10)

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid
5,73,Jacob,Klein,M,paid
6,73,Jacob,Klein,M,paid
7,73,Jacob,Klein,M,paid
9,73,Jacob,Klein,M,paid
10,73,Jacob,Klein,M,paid


*Obviously redundant information in the above list, which means we have to pay attention when inserting duplicates into the table*

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [31]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [32]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [33]:
song_select_tmp = ("""
    SELECT s.song_id, s.artist_id FROM songs_dim s JOIN artists_dim a ON s.artist_id = a.artist_id WHERE title=%s AND name=%s AND duration=%s
""")

In [34]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [39]:
for index, row in df.iterrows():
    #print(row.song, row.artist, row.length)
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    #print(results)
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # Leave out songplay_id, as it is autoincremented
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# [Optional] Example Queries

In [1]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

NameError: name 'psycopg2' is not defined

### Query 1: 

In [15]:
query_1 = """
    SELECT sp.song_id, s.title, COUNT(sp.songplay_id) AS num_of_plays
    FROM songplays_fact AS sp
    INNER JOIN songs_dim AS s ON sp.song_id = s.song_id
    GROUP BY sp.song_id, s.title
    ORDER BY COUNT(sp.songplay_id)
"""

In [16]:
cur.execute(query_1)

In [17]:
results = cur.fetchall()
results

[('SOZCTXZ12AB0182364', 'Setanta matins', 1)]

I.e. there is only one song

### Query 2: Calculate the average duration of a song

In [4]:
query_2 = """
    SELECT AVG(duration)
    FROM songs_dim
"""

In [5]:
cur.execute(query_2)

In [6]:
results = cur.fetchall()
results

[(Decimal('239.7296760563380282'),)]

When ETL'ing the full dataset, we obtain the average song duration of **239.7 seconds**.

### Query 3: 

In [7]:
query_3 = """
    SELECT s.user_id, u.last_name, u.first_name, t.year, t.month, t.day, t.hour,
    239.7 * COUNT(s.songplay_id) AS time_spent
    FROM songplays_fact AS s
    INNER JOIN users_dim AS u ON s.user_id = u.user_id
    INNER JOIN time_dim AS t ON s.start_time = t.starttime
    GROUP BY s.user_id, u.last_name, u.first_name, t.year, t.month, t.day, t.hour
    ORDER BY s.user_id, u.last_name, u.first_name, t.year, t.month, t.day, t.hour
"""

In [13]:
cur.execute(query_3)

In [14]:
results = cur.fetchmany(20)
results

[(2, 'Benjamin', 'Jizelle', 2018, 11, 6, 15, Decimal('239.7')),
 (2, 'Benjamin', 'Jizelle', 2018, 11, 6, 16, Decimal('1198.5')),
 (2, 'Benjamin', 'Jizelle', 2018, 11, 7, 15, Decimal('239.7')),
 (2, 'Benjamin', 'Jizelle', 2018, 11, 17, 10, Decimal('719.1')),
 (3, 'Valdez', 'Isaac', 2018, 11, 2, 20, Decimal('719.1')),
 (4, 'Terrell', 'Alivia', 2018, 11, 14, 3, Decimal('239.7')),
 (4, 'Terrell', 'Alivia', 2018, 11, 14, 13, Decimal('479.4')),
 (4, 'Terrell', 'Alivia', 2018, 11, 29, 3, Decimal('239.7')),
 (4, 'Terrell', 'Alivia', 2018, 11, 29, 17, Decimal('239.7')),
 (5, 'Davis', 'Elijah', 2018, 11, 27, 12, Decimal('719.1')),
 (5, 'Davis', 'Elijah', 2018, 11, 30, 19, Decimal('479.4')),
 (6, 'Owens', 'Cecilia', 2018, 11, 3, 23, Decimal('958.8')),
 (6, 'Owens', 'Cecilia', 2018, 11, 4, 4, Decimal('239.7')),
 (6, 'Owens', 'Cecilia', 2018, 11, 4, 11, Decimal('239.7')),
 (6, 'Owens', 'Cecilia', 2018, 11, 9, 2, Decimal('239.7')),
 (6, 'Owens', 'Cecilia', 2018, 11, 9, 15, Decimal('239.7')),
 (6, 'O

# Close Connection to Sparkify Database

### Drop Tables and Close Connection

In [36]:
from create_tables import drop_tables
drop_tables(cur, conn)

In [37]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.